# Ensemble of 3 classifiers

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Зададим нужные константы для тестирования:

In [2]:
# we perform T*Q-fold cross validation (T*Q-fold CV)
T = 3  # number of performing cross validations
Q = 5  # number of folds

# PATH_TO_CLASSIFIER = '../results/voting/voting_classifier.pkl'

Загрузим необходимые данные:

In [4]:
from sklearn.externals import joblib

X_train = joblib.load('../data/X_train.pkl')
y_train = joblib.load('../data/y_train.pkl')

# X_train = joblib.load('../tmp/1467_X_train.pkl')
# y_train = joblib.load('../tmp/1467_y_train.pkl')

X_test = joblib.load('../data/X_test.pkl')
y_test = joblib.load('../data/y_test.pkl')

KNN_BEST_FEATURES = [
    0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40,
    41, 42, 43, 44, 45, 46, 49, 51, 52, 53, 55, 56, 58, 60,
    62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
    81, 82, 83, 84, 86, 87, 88, 89, 91, 92, 93, 94, 95
]

display(X_train)

,DIRECTORY_ENTRY_BOUND_IMPORT/exists,DIRECTORY_ENTRY_DEBUG/exists,DIRECTORY_ENTRY_DELAY_IMPORT/exists,DIRECTORY_ENTRY_EXPORT/exists,DIRECTORY_ENTRY_IMPORT/exists,DIRECTORY_ENTRY_RESOURCE/exists,DIRECTORY_ENTRY_TLS/exists,DOSStub/entropy,DOSStub/zeros,DOS_HEADER.e_cblp,...,SECTIONS/entry_point_in_last_section,SECTIONS/entry_point_in_writeable_section,SECTIONS/executable_sections,SECTIONS/max_section_entropy,SECTIONS/rsrc_section_entropy,SECTIONS/sections_with_zero_entropy_count,SECTIONS/text_section_entropy,SECTIONS/unusual_section_names,SECTIONS/writeable_and_executable_sections,SECTIONS/writeable_sections
26682,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,0.333973,-0.507212,0.569661,-0.572900,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.377887,0.548801,0.258948,0.546524,-0.460900,-0.288560,-0.094187
50916,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,0.333973,-0.507212,0.373237,-0.234648,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.444852,0.051442,0.258948,0.525452,-0.460900,-0.288560,-0.094187
29258,-0.145786,-0.079078,-0.150130,-0.304439,-0.078087,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.268217,-0.186286,-0.665014,0.581034,-0.460900,-0.288560,-0.094187
18211,-0.145786,0.129710,1.233757,-0.304439,0.712365,0.333973,-0.507212,0.630445,-0.682722,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.252136,0.204942,-0.665014,0.586095,-0.460900,-0.288560,-0.655287
31183,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,0.333973,-0.507212,0.467182,-0.191413,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.383324,0.548744,0.258948,0.544813,-0.460900,-0.288560,-0.094187
54395,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,-2.994258,-0.507212,0.333303,-0.542699,-0.011388,...,4.395228,-0.452174,0.569053,0.459248,-2.024843,0.258948,0.809948,0.131280,0.518246,1.028014
27351,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,0.333973,-0.507212,0.421855,-0.309327,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.335065,0.369694,0.258948,0.559999,-0.460900,-0.288560,-0.094187
27791,-0.145786,-0.079078,8.845135,3.284733,1.601623,0.333973,1.971561,-1.770532,1.874648,-0.102259,...,-0.227519,-0.452174,0.569053,0.242980,1.187646,1.182910,0.534680,-0.460900,-0.288560,1.589115
19336,-0.145786,-0.079078,-0.150130,-0.304439,0.020719,0.333973,-0.507212,0.629202,-0.599746,-0.011388,...,-0.227519,-0.452174,-0.269120,1.378243,0.986296,0.258948,0.562126,-0.460900,-0.288560,-0.094187
27617,-0.145786,0.547286,-0.150130,-0.304439,-0.374507,0.333973,-0.507212,0.611762,-0.664942,-0.011388,...,-0.227519,-0.452174,-0.269120,-1.062610,-0.885523,-0.665014,0.331061,-0.460900,-0.288560,-0.655287


Загрузим готовые классификаторы:

In [5]:
knn_classifier = joblib.load('../results/knn/knn_classifier.pkl')
svm_classifier = joblib.load('../results/svm/svm_classifier.pkl')
rf_classifier = joblib.load('../results/rf/rf_classifier.pkl')

Тестирование на обучающей выборке:

In [10]:
def get_average_tn_rate(tns_df):
    return (tns_df['tn_rate'].mean(), tns_df['tn_rate'].std()*2)

def get_average_fp_rate(fps_df):
    return (fps_df['fp_rate'].mean(), fps_df['fp_rate'].std()*2)

def get_average_fn_rate(fns_df):
    return (fns_df['fn_rate'].mean(), fns_df['fn_rate'].std()*2)

def get_average_tp_rate(tps_df):
    return (tps_df['tp_rate'].mean(), tps_df['tp_rate'].std()*2)

def get_accuracy(tns_df, tps_df):
    accuracy = tps_df['tp_rate'].mean() + tns_df['tn_rate'].mean()
    deviation = (tps_df['tp_rate'] + tns_df['tn_rate']).std()*2
    return (accuracy, deviation)

In [11]:
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import confusion_matrix


def get_y_pred(X_train, y_train, X_test):
    # global knn_classifier, svm_classifier, rf_classifier
    knn_X_train = X_train.iloc[:, KNN_BEST_FEATURES]
    knn_classifier.fit(knn_X_train, y_train)
    svm_classifier.fit(X_train, y_train)
    rf_classifier.fit(X_train, y_train)
    
    knn_X_test = X_test.iloc[:, KNN_BEST_FEATURES]
    knn_y_pred = knn_classifier.predict(knn_X_test)
    svm_y_pred = svm_classifier.predict(X_test)
    rf_y_pred = rf_classifier.predict(X_test)
    assert(len(knn_y_pred) == len(svm_y_pred) == len(rf_y_pred))

    y_pred = []
    for i in range(len(knn_y_pred)):
        curr_pred = knn_y_pred[i] + svm_y_pred[i] + rf_y_pred[i]
        if curr_pred < 1.5:
            y_pred.append(0)
        else:
            y_pred.append(1)

    y_pred = np.array(y_pred)
    return y_pred



def test_classifier(X, y):
    tns, fps, fns, tps = ([], [], [], [])

    rkf = RepeatedKFold(n_splits=Q, n_repeats=T, random_state=188)
    for train_index, test_index in rkf.split(X):
        curr_X_train, curr_X_test = X.iloc[train_index], X.iloc[test_index]
        curr_y_train, curr_y_test = y.iloc[train_index], y.iloc[test_index]
        curr_y_pred = get_y_pred(curr_X_train, curr_y_train, curr_X_test)
        curr_tn, curr_fp, curr_fn, curr_tp = (
            confusion_matrix(curr_y_test, curr_y_pred).ravel()
        )

        curr_test_size = curr_tn + curr_fp + curr_fn + curr_tp
        assert(curr_test_size == len(curr_X_test) == len(curr_y_test))
        tns.append({
            'tn': curr_tn,
            'test_size': curr_test_size,
            'tn_rate': curr_tn/curr_test_size,
        })
        fps.append({
            'fp': curr_fp,
            'test_size': curr_test_size,
            'fp_rate': curr_fp/curr_test_size,
        })
        fns.append({
            'fn': curr_fn,
            'test_size': curr_test_size,
            'fn_rate': curr_fn/curr_test_size,
        })
        tps.append({
            'tp': curr_tp,
            'test_size': curr_test_size,
            'tp_rate': curr_tp/curr_test_size,
        })

    tns_df = pd.DataFrame(tns)
    fps_df = pd.DataFrame(fps)
    fns_df = pd.DataFrame(fns)
    tps_df = pd.DataFrame(tps)
    return (tns_df, fps_df, fns_df, tps_df)


tns_df, fps_df, fns_df, tps_df = (
    test_classifier(X_train, y_train)
)

print('TRAIN accuracy =', get_accuracy(tns_df, tps_df))
print('TRAIN fp rate =', get_average_fp_rate(fps_df))
print('TRAIN tp rate =', get_average_tp_rate(tps_df))
print('TRAIN tn rate =', get_average_tn_rate(tns_df))
print('TRAIN fn rate =', get_average_fn_rate(fns_df))

/home/artem/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/artem/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/artem/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/artem/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/hom

TRAIN accuracy = (0.956264200466526, 0.0043206636492035145)
TRAIN fp rate = (0.020282116488506915, 0.0037430854145039435)
TRAIN tp rate = (0.48130384161771755, 0.009789996696679506)
TRAIN tn rate = (0.47496035884880855, 0.011266190030329542)
TRAIN fn rate = (0.023453683044966773, 0.003399612844187939)


Тестирование на тестовой выборке:

In [12]:
knn_X_test = X_test.iloc[:, KNN_BEST_FEATURES]

knn_y_pred = knn_classifier.predict(knn_X_test)
svm_y_pred = svm_classifier.predict(X_test)
rf_y_pred = rf_classifier.predict(X_test)
assert(len(knn_y_pred) == len(svm_y_pred) == len(rf_y_pred))

y_pred = []
for i in range(len(knn_y_pred)):
    curr_pred = knn_y_pred[i] + svm_y_pred[i] + rf_y_pred[i]
    if curr_pred < 1.5:
        y_pred.append(0)
    else:
        y_pred.append(1)

y_pred = np.array(y_pred)

assert(len(y_test) == len(y_pred))
test_tn, test_fp, test_fn, test_tp = (
    confusion_matrix(y_test, y_pred).ravel()
)

print('TEST accuracy =', (test_tp + test_tn)/len(y_pred))
print('TEST tn rate =', test_tn/len(y_pred))
print('TEST fp rate =', test_fp/len(y_pred))
print('TEST fn rate =', test_fn/len(y_pred))
print('TEST tp rate =', test_tp/len(y_pred))

TEST accuracy = 0.955241329399
TEST tn rate = 0.479151589378
TEST fp rate = 0.020152535768
TEST fn rate = 0.0246061348327
TEST tp rate = 0.476089740021
